In [1]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
from datetime import datetime

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw

In [2]:
# Load data
data = pd.read_csv(os.getcwd() + '/data_storage/data_11000000_from_simulation_mix_09_12_18_18_20_50.csv')

In [3]:
# Some cleaning of the data
data = data[['v', 'a', 'w', 'rt', 'choice', 'nf_likelihood']]
data = data.loc[data['w'] > 0.1]
data = data.loc[data['w'] < 0.9]
data = data.loc[data['a'] > 0.5]

mini_data = data.loc[1:10000]


train_f, train_l, test_f, test_l = mdw.train_test_split_rt_choice(data = data,
                                                                  write_to_file = False,
                                                                  from_file = False,
                                                                  p_train = 0.8,
                                                                  backend = 'keras')
# Choice probabilities
# train_f, train_l, test_f, test_l = mdw.train_test_from_file_choice_probabilities(n_samples = 2500000,
#                                                             f_signature = '_choice_probabilities_analytic_',
#                                                                                 backend = 'keras')

# rt_choice
# train_f, train_l, test_f, test_l = mdw.train_test_from_file_rt_choice(n_samples = 11000000,
#                                                                       f_signature = '_from_simulation_mix_',
#                                                                       backend = 'keras')

In [4]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()
cpm.data['train_features'] = train_f
cpm.data['train_labels'] = train_l
cpm.data['test_features'] = test_f
cpm.data['test_labels'] = test_l

In [5]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

{'input_shape': 3,
 'output_shape': 1,
 'output_activation': 'sigmoid',
 'hidden_layers': [20, 20, 20, 20],
 'hidden_activations': ['relu', 'relu', 'relu', 'relu'],
 'l1_activation': [0.0, 0.0, 0.0, 0.0],
 'l2_activation': [0.0, 0.0, 0.0, 0.0],
 'l1_kernel': [0.0, 0.0, 0.0, 0.0],
 'l2_kernel': [0.0, 0.0, 0.0, 0.0],
 'optimizer': 'Nadam',
 'loss': 'mse',
 'metrics': ['mse']}

In [6]:
# Parameters governing training
cpm.train_params

{'callback_funs': ['ReduceLROnPlateau', 'EarlyStopping', 'ModelCheckpoint'],
 'plateau_patience': 10,
 'min_delta': 0.0001,
 'early_stopping_patience': 15,
 'callback_monitor': 'loss',
 'min_learning_rate': 1e-07,
 'red_coef_learning_rate': 0.1,
 'ckpt_period': 10,
 'ckpt_save_best_only': True,
 'ckpt_save_weights_only': True,
 'max_train_epochs': 2000,
 'batch_size': 10000,
 'warm_start': False,
 'checkpoint': 'ckpt',
 'model_cnt': 0}

In [7]:
# Parameters concerning data storage
cpm.data_params

{'data_type': 'choice_probabilities',
 'model_directory': '/home/afengler/git_repos/nn_likelihoods/keras_models',
 'checkpoint': 'ckpt',
 'model_name': 'dnnregressor',
 'data_type_signature': '_choice_probabilities_analytic_',
 'timestamp': '09_19_18_23_17_39',
 'training_data_size': 2500000}

In [8]:
# SPECIFYING META PARAMETERS THAT STAY CONSTANT DURING HYPERPARAMETER OPTIMIZATION

# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['input_shape'] = 5

# Training params
# Meta
cpm.train_params['early_stopping_patience'] = 5
cpm.train_params['plateau_patience'] = 3
cpm.train_params['min_delta'] = 0.002
cpm.train_params['ckpt_period'] = 1
cpm.train_params['model_cnt'] = 0
cpm.train_params['max_train_epochs'] = 50

# Hyper
#cpm.train_params['l1_kernel']
cpm.model_params['hidden_layers'] = [5, 5, 5, 5]
#cpm.train_params['hidden_activations']
#cpm.train_params['l2_kernel'] = [0.5, 0.5, 0.5, 0.5]
#cpm.train_params['l2_activation'] = [0.5, 0.5, 0.5, 0.5]

# Data params
cpm.data_params['data_type'] = 'wfpt'
cpm.data_params['data_type_signature'] = '_choice_rt_'
cpm.data_params['training_data_size'] = 11000000

# Update timestamp
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')

In [9]:
# Make model
# cpm.keras_model_generate(save_model = True)

In [10]:
# Train model
# cpm.run_training(save_history = True, 
#                  warm_start = False)

In [ ]:
# Hyperparameter training loop:

# Runs: 
num_runs = 20
cnt = 0
max_layers = 5
layer_sizes = [10, 50, 60]
batch_sizes = [1000, 1000]
regularization_sizes = [0.0, 0.0]

# Update model directory to make sure we collect all our models from this hyperparameter optimization run in the same place
cpm.data_params['model_directory'] =  '/home/afengler/git_repos/nn_likelihoods/keras_models/'
cpm.data_params['model_name'] = 'dnnregressor_wftp_hyp_opt_post_stability_experiment'
cpm.train_params['model_cnt'] = 0

histories = []

while cnt < num_runs:
    cnt += 1
    
    # Sample # layers 
    #num_layers = np.random.choice(np.arange(4, max_layers + 1, 1))
    num_layers = 4
    # Layer sizes
    layers = [10, 50, 50, 50]
    activations = ['relu', 'relu', 'relu', 'relu']
    regularizers_l1 = [0.0, 0.0, 0.0, 0.0]
    regularizers_l2 = [0.0, 0.0, 0.0, 0.0]
    regularizer = np.random.choice(['l1', 'l2', 'none'])
    regularizer_size = np.random.choice(regularization_sizes)
    
#     for i in range(0, num_layers, 1):
#         layers.append(np.random.choice(layer_sizes))
#         activations.append('relu')
#         if regularizer == 'l1':
#             regularizers_l1.append(regularizer_size)
#             regularizers_l2.append(0.0)
#         if regularizer == 'l2':
#             regularizers_l1.append(0.0)
#             regularizers_l2.append(regularizer_size)
#         else:
#             regularizers_l1.append(0.0)
#             regularizers_l2.append(0.0)
        
    # Batch size
    batch_size = np.random.choice(batch_sizes)
    
    # Update relevant model parameters
    cpm.train_params['batch_size'] = batch_size
    print('batch_size: ', batch_size)
    cpm.model_params['hidden_layers'] = layers
    print('layers: ', layers)
    cpm.model_params['hidden_activations'] = activations
    print('hidden_activations:', activations)
#     cpm.model_params['l1_activation'] = regularizers_l1
#     print('l1_activatons: ', regularizers_l1)
#     cpm.model_params['l2_activation'] = regularizers
#     print('l2_activations:', regularizers_l2)
    cpm.model_params['l1_kernel'] = regularizers_l1
    print('l1_kernel: ', regularizers_l1)
    cpm.model_params['l2_kernel'] = regularizers_l2
    print('l2_kernel: ', regularizers_l2)
    
    # Make new timestamp
    #cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
    
    # Make model
    cpm.keras_model_generate(save_model = True)
    
    # Train model
    cpm.run_training(save_history = True, 
                     warm_start = False) # Note that this increments model count automatically !
    
#     histories[-1]['model_cnt'] = cpm.train_params['model_cnt']
#     histories[-1]['num_layers'] = num_layers
#     histories[-1]['size_layers'] = str(layers)
#     histories[-1]['activations'] = str(activations) 
#     histories[-1]['batch_size'] = batch_size
    
    print(cnt)
    
# histories = pd.concat(histories)
# histories['optimizer'] = cpm.model_params['optimizer']
# histories['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
# histories.to_csv(cpm.data_params['model_directory'] + cpm.data_params['model_name'] + '_choice_rt_' +\
#                  cpm.data_params['timestamp'] + '/hyp_opt_histories.csv')

batch_size:  1000
layers:  [10, 50, 50, 50]
hidden_activations: ['relu', 'relu', 'relu', 'relu']
l1_kernel:  [0.0, 0.0, 0.0, 0.0]
l2_kernel:  [0.0, 0.0, 0.0, 0.0]
Train on 6069260 samples, validate on 1518747 samples
Epoch 1/50
6069260/6069260 [==============================] - 236s 39us/step - loss: 2.4588 - mean_squared_error: 2.4588 - val_loss: 0.9254 - val_mean_squared_error: 0.9254

Epoch 00001: val_loss improved from inf to 0.92544, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_0_01
Epoch 2/50
6069260/6069260 [==============================] - 273s 45us/step - loss: 1.1166 - mean_squared_error: 1.1166 - val_loss: 3.0979 - val_mean_squared_error: 3.0979

Epoch 00002: val_loss did not improve from 0.92544
Epoch 3/50
6069260/6069260 [==============================] - 277s 46us/step - loss: 0.7893 - mean_squared_error: 0.7893 - val_loss: 0.4891 - val_mean_squared_error: 0.4891

6069260/6069260 [==============================] - 275s 45us/step - loss: 0.2503 - mean_squared_error: 0.2503 - val_loss: 0.1595 - val_mean_squared_error: 0.1595

Epoch 00030: val_loss improved from 0.16041 to 0.15950, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_0_30
Epoch 31/50
6069260/6069260 [==============================] - 282s 46us/step - loss: 0.2574 - mean_squared_error: 0.2574 - val_loss: 0.1330 - val_mean_squared_error: 0.1330

Epoch 00031: val_loss improved from 0.15950 to 0.13296, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_0_31
Epoch 32/50
6069260/6069260 [==============================] - 282s 46us/step - loss: 0.2511 - mean_squared_error: 0.2511 - val_loss: 0.2073 - val_mean_squared_error: 0.2073

Epoch 00032: val_loss did not improve from 0.13296
Epoc

6069260/6069260 [==============================] - 285s 47us/step - loss: 0.5200 - mean_squared_error: 0.5200 - val_loss: 0.6219 - val_mean_squared_error: 0.6219

Epoch 00007: val_loss did not improve from 0.29078
Epoch 8/50
6069260/6069260 [==============================] - 277s 46us/step - loss: 0.5072 - mean_squared_error: 0.5072 - val_loss: 1.7539 - val_mean_squared_error: 1.7539

Epoch 00008: val_loss did not improve from 0.29078
Epoch 9/50
6069260/6069260 [==============================] - 282s 46us/step - loss: 0.4731 - mean_squared_error: 0.4731 - val_loss: 0.3900 - val_mean_squared_error: 0.3900

Epoch 00009: val_loss did not improve from 0.29078
Epoch 10/50
6069260/6069260 [==============================] - 284s 47us/step - loss: 0.4505 - mean_squared_error: 0.4505 - val_loss: 0.4308 - val_mean_squared_error: 0.4308

Epoch 00010: val_loss did not improve from 0.29078
Epoch 11/50
6069260/6069260 [==============================] - 276s 45us/step - loss: 0.4179 - mean_squared_er

6069260/6069260 [==============================] - 212s 35us/step - loss: 0.2293 - mean_squared_error: 0.2293 - val_loss: 0.1307 - val_mean_squared_error: 0.1307

Epoch 00041: val_loss improved from 0.13300 to 0.13072, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_1_41
Epoch 42/50
6069260/6069260 [==============================] - 203s 33us/step - loss: 0.2239 - mean_squared_error: 0.2239 - val_loss: 0.1470 - val_mean_squared_error: 0.1470

Epoch 00042: val_loss did not improve from 0.13072
Epoch 43/50
6069260/6069260 [==============================] - 211s 35us/step - loss: 0.2198 - mean_squared_error: 0.2198 - val_loss: 0.1234 - val_mean_squared_error: 0.1234

Epoch 00043: val_loss improved from 0.13072 to 0.12341, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_1_43
Epoc

6069260/6069260 [==============================] - 207s 34us/step - loss: 0.2996 - mean_squared_error: 0.2996 - val_loss: 0.1801 - val_mean_squared_error: 0.1801

Epoch 00021: val_loss improved from 0.20652 to 0.18005, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_2_21
Epoch 22/50
6069260/6069260 [==============================] - 215s 35us/step - loss: 0.3009 - mean_squared_error: 0.3009 - val_loss: 0.2935 - val_mean_squared_error: 0.2935

Epoch 00022: val_loss did not improve from 0.18005
Epoch 23/50
6069260/6069260 [==============================] - 206s 34us/step - loss: 0.2719 - mean_squared_error: 0.2719 - val_loss: 0.1230 - val_mean_squared_error: 0.1230

Epoch 00023: val_loss improved from 0.18005 to 0.12297, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_2_23
Epoc

...

6069260/6069260 [==============================] - 273s 45us/step - loss: 0.3587 - mean_squared_error: 0.3587 - val_loss: 0.2170 - val_mean_squared_error: 0.2170

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.

Epoch 00015: val_loss did not improve from 0.16492
Epoch 16/50
6069260/6069260 [==============================] - 274s 45us/step - loss: 0.1023 - mean_squared_error: 0.1023 - val_loss: 0.1060 - val_mean_squared_error: 0.1060

Epoch 00016: val_loss improved from 0.16492 to 0.10601, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_16_16
Epoch 17/50
6069260/6069260 [==============================] - 275s 45us/step - loss: 0.0925 - mean_squared_error: 0.0925 - val_loss: 0.0885 - val_mean_squared_error: 0.0885

Epoch 00017: val_loss improved from 0.10601 to 0.08845, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregresso

6069260/6069260 [==============================] - 272s 45us/step - loss: 0.0588 - mean_squared_error: 0.0588 - val_loss: 0.0739 - val_mean_squared_error: 0.0739

Epoch 00040: val_loss did not improve from 0.05422
Epoch 41/50
6069260/6069260 [==============================] - 272s 45us/step - loss: 0.0585 - mean_squared_error: 0.0585 - val_loss: 0.0559 - val_mean_squared_error: 0.0559

Epoch 00041: val_loss did not improve from 0.05422
Epoch 42/50
6069260/6069260 [==============================] - 272s 45us/step - loss: 0.0578 - mean_squared_error: 0.0578 - val_loss: 0.0559 - val_mean_squared_error: 0.0559

Epoch 00042: val_loss did not improve from 0.05422
Epoch 43/50
6069260/6069260 [==============================] - 272s 45us/step - loss: 0.0571 - mean_squared_error: 0.0571 - val_loss: 0.0678 - val_mean_squared_error: 0.0678

Epoch 00043: val_loss did not improve from 0.05422
Epoch 44/50
6069260/6069260 [==============================] - 272s 45us/step - loss: 0.0570 - mean_squared_

6069260/6069260 [==============================] - 274s 45us/step - loss: 0.2897 - mean_squared_error: 0.2897 - val_loss: 0.2497 - val_mean_squared_error: 0.2497

Epoch 00018: val_loss did not improve from 0.12317
Epoch 19/50
6069260/6069260 [==============================] - 274s 45us/step - loss: 0.2837 - mean_squared_error: 0.2837 - val_loss: 0.1606 - val_mean_squared_error: 0.1606

Epoch 00019: val_loss did not improve from 0.12317
Epoch 20/50
6069260/6069260 [==============================] - 275s 45us/step - loss: 0.2744 - mean_squared_error: 0.2744 - val_loss: 0.4134 - val_mean_squared_error: 0.4134

Epoch 00020: val_loss did not improve from 0.12317
Epoch 21/50
6069260/6069260 [==============================] - 276s 45us/step - loss: 0.2678 - mean_squared_error: 0.2678 - val_loss: 0.2658 - val_mean_squared_error: 0.2658

Epoch 00021: val_loss did not improve from 0.12317
Epoch 22/50
6069260/6069260 [==============================] - 274s 45us/step - loss: 0.2812 - mean_squared_

6069260/6069260 [==============================] - 275s 45us/step - loss: 0.0550 - mean_squared_error: 0.0550 - val_loss: 0.0515 - val_mean_squared_error: 0.0515

Epoch 00046: val_loss improved from 0.05290 to 0.05150, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_post_stability_experiment_choice_rt_09_19_18_23_17_40/ckpt_17_46
Epoch 47/50
6069260/6069260 [==============================] - 275s 45us/step - loss: 0.0542 - mean_squared_error: 0.0542 - val_loss: 0.0516 - val_mean_squared_error: 0.0516

Epoch 00047: val_loss did not improve from 0.05150
Epoch 48/50
6069260/6069260 [==============================] - 274s 45us/step - loss: 0.0537 - mean_squared_error: 0.0537 - val_loss: 0.0522 - val_mean_squared_error: 0.0522

Epoch 00048: val_loss did not improve from 0.05150
Epoch 49/50
6069260/6069260 [==============================] - 275s 45us/step - loss: 0.0533 - mean_squared_error: 0.0533 - val_loss: 0.0503 - val_mean_squared_error: 0

6069260/6069260 [==============================] - 223s 37us/step - loss: 0.2808 - mean_squared_error: 0.2808 - val_loss: 0.1591 - val_mean_squared_error: 0.1591

Epoch 00022: val_loss did not improve from 0.15283
Epoch 23/50
6069260/6069260 [==============================] - 222s 37us/step - loss: 0.2769 - mean_squared_error: 0.2769 - val_loss: 0.1797 - val_mean_squared_error: 0.1797

Epoch 00023: val_loss did not improve from 0.15283
Epoch 24/50
6069260/6069260 [==============================] - 224s 37us/step - loss: 0.2756 - mean_squared_error: 0.2756 - val_loss: 0.1542 - val_mean_squared_error: 0.1542

Epoch 00024: val_loss did not improve from 0.15283
Epoch 25/50
6069260/6069260 [==============================] - 224s 37us/step - loss: 0.2726 - mean_squared_error: 0.2726 - val_loss: 0.3023 - val_mean_squared_error: 0.3023

Epoch 00025: val_loss did not improve from 0.15283
Epoch 26/50
6069260/6069260 [==============================] - 223s 37us/step - loss: 0.2641 - mean_squared_